In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

# Streaming = we can check response of model
# callbacks = allow you to see responses as it is being generated
chat = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients.",
        ),
        ("human", "I want to cook {cuisine} food."),
    ],
)

chef_chain = chef_prompt | chat

In [ ]:
veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a vegetarian chef specialized on making traditional recipes vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to recipe it.",
        ),
        ("human", "{recipe}"),
    ]
)

veg_chef_chain = veg_chef_prompt | chat

final_chain = {"recipe" : chef_chain} | veg_chef_chain

final_chain.invoke({
    "cuisine" : "indian"
})

In [ ]:
from typing import Any, Dict, List
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector



chat = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

# FewShotPromptTemplate : 템플릿을 직접 만드는 것 보다, 예시를 들어 보여주는 것이 더 쉬움
# 왜냐하면 모델이 텍스트를 만들어낼 것이기 때문 - 이것을 통해 예제를 형식화 할 수 있음
# 그리고 이 방식이 LLM을 만드는 방식이고, 더 나은 LLM을 만들 수 있음

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples
        
    def add_example(self, example):
        self.examples.append(example)
    
    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]


example_prompt = PromptTemplate.from_template("Human: {question}\nAI : {answer}")

# LengthBasedExampleSelector : can limit amount of examples - save money!
example_selector = RandomExampleSelector(
    examples= examples,
)


prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)

prompt.format(country= "Germany")



In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts import load_prompt

# how to load prompt tamplates from disk

prompt = load_prompt("./prompt.yaml")


chat = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

prompt.format(country="xxx")

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.pipeline import PipelinePromptTemplate

# how to compose many prompt

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)
intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
    """
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
    """
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
    """
)

final = PromptTemplate.from_template(
    """
    {intro}

    {example}

    {start}
    """
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]

full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)

chain = full_prompt | chat

chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location",
        "example_answer": "Arrrrr! That is a secret, ARRRRR",
        "question": "What is your favorate food?",
    }
)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
# from langchain.globals import set_llm_cache, set_debug
# from langchain.cache import InMemoryCache, SQLiteCache

#set_llm_cache : save the responses of Language Model(LM)
# for same question, save the answer and repeat it
set_llm_cache(SQLiteCache("cache.db"))
set_debug(True)
chat = ChatOpenAI(
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler(),
    # ],
)

chat.predict("How do you make italian pasta??")

In [ ]:
chat.predict("How do you make italian pasta??")

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

chat = ChatOpenAI(
    temperature=0.1,
)

with get_openai_callback() as usage :
    a = chat.predict("What is recipe for suju?")
    b = chat.predict("What is recipe for bread?")
    print(a,b,"\n")
    print(usage)

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: What is recipe for suju?"
  ]
}
[llm/end] [1:llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Suju is a traditional Korean alcoholic beverage made from fermented rice. Here is a simple recipe to make suju at home:\n\nIngredients:\n- 2 cups short-grain rice\n- 1 cup nuruk (fermentation starter)\n- 8 cups water\n\nInstructions:\n1. Rinse the rice in cold water until the water runs clear. Drain the rice and let it sit for 30 minutes to absorb some water.\n2. Steam the rice until it is fully cooked and slightly sticky. Let the rice cool to room temperature.\n3. In a large bowl, mix the cooked rice with the nuruk until well combined.\n4. Transfer the rice mixture to a large glass jar or crock. Add water and mix well.\n5. Cover the jar with a clean cloth and let it ferment in a dark, cool place for 7-10 days. Stir the mixture every day to ensure even fer

In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.llms.openai import OpenAI
from langchain.llms.loading import load_llm
# chat = OpenAI(temperature=0.1, max_tokens=450, model="gpt-3.5-turbo")

chat = load_llm("model.json")

chat


/Users/yumingi/Desktop/Coding/FULLSTACK-GPT/env/lib/python3.11/site-packages/langchain/llms/openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/yumingi/Desktop/Coding/FULLSTACK-GPT/env/lib/python3.11/site-packages/langchain/llms/openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


OpenAIChat(client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_kwargs={'temperature': 0.1, 'max_tokens': 450, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'request_timeout': None, 'logit_bias': {}})